In [1]:
import os
import copy
import random
import math

In [36]:
class City:
    
    def __init__(self, city):
        self.number = city[0]
        self.x = city[1]
        self.y = city[2]
        
    def __repr__(self):
        return f'({self.x},{self.y})'
        
    def __str__(self):
        return f'City No:{self.number}, X:{self.x}, Y:{self.y}'


class TSP:
    def __init__(self, path) -> None:
        self.cities = []
        self.city_dict = {}
        self.opt_tour = []
        self.init_tsp(path)
        
    def init_tsp(self, path):
        
        # Populate initial configuration of cities
        file = open(f'tsp_files/vlsi/{path}/{path}.tsp').readlines()
        
        self.name = file[0]
        self.size = int(file[5].split(" ")[-1].replace('\n', ''))
        
        for city_line in file[8:-1]:
            city = city_line.split(" ")
            city[1] = int(city[1])
            city[2] = int(city[2].replace('\n', ''))
            
            self.cities.append(City(city))
            
            self.city_dict[city[0]] = City(city)
            
        self.cities.append(self.cities[0])
        
        
        # Store data on optimal path
        file = open(f'tsp_files/vlsi/{path}/{path}.tour').readlines()
        
        self.opt_tour_length = int(file[1].split(" ")[-1].replace("\n", ""))
        
        for city_line in file[5:-2]:
            city_num = city_line.replace("\n", "")
            self.opt_tour.append(self.city_dict[city_num])
        
    def set_cities(self, cities):
        self.cities = cities
        
    def get_cities(self):
        return copy.deepcopy(self.cities)

In [90]:
class Chromosome:
    
    def __init__(self, cities):
        
        self.cities = cities
        self.fitness = None
        
    def __repr__(self):
        return f"(Fitness: {self.fitness}, {self.cities[:2]} ... {self.cities[-2:]})"
    
    def __str__(self):
        return f"Fitness: {self.fitness}\n{self.cities}"
    
    def set_fitness(self, fitness):
        self.fitness = fitness

class GeneticAlgorithm:
    
    def __init__(self, pop_size, cross_prop, tsp):
        self.POP_SIZE = pop_size
        self.cross_prop = cross_prop
        self.population = []
        
        self.init_population(tsp)
        
    def init_population(self, tsp):
        
        for i in range(self.POP_SIZE):
            temp_tsp = copy.deepcopy(tsp)
            temp_city_list = temp_tsp.get_cities()
            start_city = temp_city_list[0]
            temp_city_list = temp_city_list[1:-1]
            
            random.shuffle(temp_city_list)
            temp_city_list.insert(0, start_city)
            temp_city_list.insert(len(temp_city_list), start_city)
            
#             temp_tsp.set_cities(temp_city_list)

            chrm = Chromosome(temp_city_list)
            
            self.population.append(chrm)
            
    def eval_fitness(self, indv):
        
        total_fitness = 0
        
        for i in range(len(indv.cities)-1):
            total_fitness += math.dist([indv.cities[i].x, indv.cities[i].y], [indv.cities[i+1].x, indv.cities[i+1].y])
            
        return round(total_fitness)
    
    def roulette_selection(self, pop):
        total_fitness = sum([1/x.fitness for x in pop])
        prob_list = [round((1/x.fitness)/total_fitness, 2) for x in pop]
        
        rw_rand_num = round(random.uniform(0.0, sum(prob_list)), 2)

        curr_wheel_fitness = 0

        for i, indv_prob in enumerate(prob_list):
            curr_wheel_fitness += indv_prob

            if rw_rand_num < curr_wheel_fitness:
                return i
    
    def book_selection(self, pop, sel_size):
        # Book's implementation of selection
        
        sel_pop = []
        
        while len(sel_pop) < sel_size:
            sel_idx = self.roulette_selection(pop)
            sel_pop.append(pop.pop(sel_idx))
                    
        return sel_pop
    
    def book_crossover(self, pop, cross_size):
        # Book's implementation of crossover
        # Single Point Crossover
        
        cross_pop = []
        
        p1_idx = self.roulette_selection(pop)
        p2_idx = self.roulette_selection(pop)
        while p1_idx == p2_idx:
            p2_idx = self.roulette_selection(pop)
        
        start_point = pop[0].cities[0]
        parent1 = pop[p1_idx].cities[1:-1]
        parent2 = pop[p2_idx].cities[1:-1]
        
        print(start_point)
        print(parent1)
        print(parent2)
        
        cross_point = random.randint(0,len(parent1)-3)
        
        
            
    def fit(self):
        
        for indv in self.population:
            indv.set_fitness(self.eval_fitness(indv))
            
        self.population.sort(key= lambda x: x.fitness)
        
        temp_pop = copy.deepcopy(self.population)
        # Definies the amount of chromosomes that will be selected. the last term ensures the size of the crossover population is a multiple of 2 as crossover produces pairs.
        sel_size = self.POP_SIZE - (round(self.cross_prop*self.POP_SIZE) + round(self.cross_prop*self.POP_SIZE)%2)
        
        sel_pop = self.book_selection(temp_pop, sel_size)
        
        temp_pop = copy.deepcopy(self.population)
        cross_size = (round(self.cross_prop*self.POP_SIZE) + round(self.cross_prop*self.POP_SIZE)%2)
        self.book_crossover(temp_pop, cross_size)
        
        # print(temp_pop)

In [91]:
path = "xqf131"
t1 = TSP(path)

ga = GeneticAlgorithm(10, 0.5, t1)

In [92]:
ga.fit()

City No:1, X:0, Y:13
[(41,34), (84,38), (74,24), (25,15), (18,39), (25,26), (34,41), (25,28), (78,35), (28,16), (84,34), (38,20), (51,47), (5,25), (71,16), (15,25), (0,39), (18,17), (74,35), (18,23), (81,17), (10,10), (18,31), (28,43), (18,11), (25,24), (80,5), (18,33), (78,32), (32,31), (80,10), (74,16), (18,25), (25,29), (61,45), (33,31), (28,28), (34,26), (35,31), (38,34), (107,27), (77,21), (35,17), (51,45), (40,22), (18,19), (2,0), (71,13), (18,29), (79,37), (15,37), (5,19), (8,0), (63,6), (78,10), (15,43), (38,16), (34,15), (12,10), (12,5), (57,25), (15,8), (25,11), (15,31), (56,25), (33,26), (61,47), (34,31), (48,6), (5,43), (5,31), (41,23), (74,39), (74,29), (33,15), (84,29), (28,20), (71,47), (11,10), (18,27), (18,42), (9,10), (18,35), (18,37), (71,11), (34,29), (28,30), (18,15), (57,12), (84,24), (25,9), (0,27), (74,12), (74,20), (28,47), (15,13), (18,21), (18,13), (28,40), (25,23), (32,26), (64,22), (5,13), (5,37), (48,22), (41,35), (84,20), (57,44), (78,39), (28,34), (15,19